ControlNet
=====

**Adding Conditional Control to Text-to-Image Diffusion Models**

 * Paper: https://arxiv.org/pdf/2302.05543

![ControlNet Overview](../assets/controlnet_overview.png)

![ControlNet Overview](../assets/controlnet_arch.png)